In [23]:
import pandas as pd
from datetime import datetime

def processLifts(row, columns):

    metaRow = processDates(row, columns)
    #https://stackoverflow.com/a/57001947
    rowDict = {columns[0]:metaRow["Date"]} # add date
    rowLifts, orderList = {}, []
    
    for i in range(2,len(row)): # columns (no date or location)
        if pd.isna(row[i]):
            continue # no lift available
        else:
            cell = row[i]
            num, sets = "", 1
            for j in range(len(cell)): # characters
                if j == len(cell)-1 and cell[j].isdigit(): # last index ending with number
                    num += cell[j]
                    rowDict[columns[i]+" Set %s" % sets] = int(num)
                    rowLifts[columns[i]+ " Order"] = -1
                elif j == len(cell)-1:
                    rowLifts[columns[i]+ "Order"] = -1
                elif cell[j].isdigit():
                    num += cell[j] # keep adding numbers
                elif ":" in cell[j]:
                    rowDict[columns[i]+" Reps Set %s" % sets] = int(num) # reps for current and future sets unless "-"
                    num = ""
                elif "," in cell[j] or "-" in cell[j]:
                    rowDict[columns[i]+" Set %s" % sets] = int(num)
                    num = ""
                    sets += 1
                elif ";" in cell[j] and j+1 < len(cell): # no out of index entry
                    orderNum = int(cell[j+1:])
                    rowLifts[columns[i]+ " Order"] = orderNum
                    orderList.append(orderNum)
                    break # reached last character
                    
    #https://stackoverflow.com/a/34045983
    for key, value in rowLifts.items():
        if value == -1:
            rowLifts[key] = (set([i for i in range(1, len(rowLifts)+1)]) - set(orderList)).pop()
            
    rowDict.update(rowLifts) 
    return (rowDict, metaRow)

def processDates(row, columns):
    rowDate = datetime.strptime(row[0], "%m/%d/%Y")
    rowLocation = row[1].split("-")
    if rowLocation[0] == "H":
        rowLocation[0] = "Home"
    elif rowLocation[0] == "U":
        rowLocation[0] = "University"
    return {columns[0]:rowDate, "Day":rowDate.strftime("%A"), "Place":rowLocation[0], "City":rowLocation[1]}
                
df = pd.read_excel("Training Log.ods")
columns = ["Date","Location","Back Squat","Bench Press","Deadlift","Overhead Press","Bent Over Rows","Romanian Deadlift"]
#https://stackoverflow.com/a/55557758
resultList = [processLifts(row, columns) for row in df[columns].to_numpy()]
rowList, metaList = [], []
for row in resultList:
    rowList.append(row[0])
    metaList.append(row[1])
pd.DataFrame.from_dict(rowList)
#pd.DataFrame.from_dict(metaList)


Date  Deadlift Reps Set 1  Deadlift Set 1  Deadlift Order  \
0    2020-12-29 00:00:00                  6.0           115.0             1.0   
1    2021-01-03 00:00:00                  1.0           135.0             1.0   
2    2021-01-06 00:00:00                  NaN             NaN             NaN   
3    2021-01-08 00:00:00                  2.0           135.0             1.0   
4    2021-01-13 00:00:00                  NaN             NaN             NaN   
..                   ...                  ...             ...             ...   
133  2024-04-04 00:00:00                  6.0           135.0             1.0   
134  2024-04-30 00:00:00                  NaN             NaN             NaN   
135  2024-05-02 00:00:00                  6.0           135.0             3.0   
136  2024-05-06 00:00:00                  6.0           185.0             NaN   
137  2024-09-09 00:00:00                  NaN             NaN             NaN   

     Back Squat Reps Set 1  Back Squat Set 1  Back Squat Order  \
0                      NaN               NaN               NaN   
1                      NaN               NaN               NaN   
2                      7.0             135.0               1.0   
3                      NaN               NaN               NaN   
4                      8.0             135.0               1.0   
..                     ...               ...               ...   
133                    6.0             185.0               3.0   
134                    NaN               NaN               NaN   
135                    6.0             135.0               1.0   
136                    6.0             135.0               3.0   
137                    8.0             135.0               1.0   

     Deadlift Set 2  Bench Press Reps Set 1  Bench Press Set 1  ...  \
0               NaN                     NaN                NaN  ...   
1               NaN                     NaN                NaN  ...   
2               NaN                     NaN                NaN  ...   
3               NaN                     NaN                NaN  ...   
4               NaN                     NaN                NaN  ...   
..              ...                     ...                ...  ...   
133           135.0                     6.0              135.0  ...   
134             NaN                     NaN                NaN  ...   
135           135.0                     6.0              135.0  ...   
136           185.0                     6.0              135.0  ...   
137             NaN                     8.0              135.0  ...   

     Bent Over Rows Order  Overhead Press Set 3  Bent Over Rows Reps Set 3  \
0                     NaN                   NaN                        NaN   
1                     NaN                   NaN                        NaN   
2                     NaN                   NaN                        NaN   
3                     NaN                   NaN                        NaN   
4                     NaN                   NaN                        NaN   
..                    ...                   ...                        ...   
133                   NaN                   NaN                        NaN   
134                   1.0                   NaN                        NaN   
135                   NaN                   NaN                        NaN   
136                   NaN                   NaN                        NaN   
137                   NaN                   NaN                        NaN   

     Bent Over Rows Set 3  Bench PressOrder  Bent Over Rows Set 4  \
0                     NaN               NaN                   NaN   
1                     NaN               NaN                   NaN   
2                     NaN               NaN                   NaN   
3                     NaN               NaN                   NaN   
4                     NaN               NaN                   NaN   
..                    ...               ...       